# Deep learning

In [1]:
import torch
import torchvision

from tqdm.auto import tqdm
import numpy as np


DATA_DIR = "data/office"
VAL_SPLIT = .2
RANDOM_SEED = 42
BATCH_SIZE = 16

In [2]:
import sklearn.preprocessing
import imageio.v2 as imageio

import os
from typing import Callable


def resnet_preprocessor(image: np.ndarray) -> np.ndarray:
    """
    Preprocesses an image for ResNet model.

    :param numpy.ndarray image: The input image.
    :return: Preprocessed image.
    :rtype: numpy.ndarray
    """
    # assume RGB dimension, either as first or last dimension
    #assert image.shape[0] == 3 or image.shape[-1] == 3

    #if image.shape[-1] == 3:
        #image = np.moveaxis(image, -1, 0)
        #print(image.shape)

    preprocess = torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
            ),
        ]
    )

    image = preprocess(image)
    image = image.unsqueeze(0) #mini-batch creation
    return image


class ImageDataset(torch.utils.data.Dataset):
    """
    Lazily loads images from a root directory.
    Directory is assumed to be of shape "<root>/<class_name>/<instance_file>".
    Allows custom functions for reading, preprocessing each image and setting the label encodings.
    """

    def __init__(
        self,
        data_dir: str,
        parser_func: Callable = imageio.imread,
        preprocessing_func: Callable[[np.ndarray], np.ndarray] = resnet_preprocessor,
        label_encoder=None,
    ):
        """
        Initializes the ImageDataset.

        :param str data_dir: Root directory containing the dataset.
        :param parser_func: Function to parse images.
        :type parser_func: Callable, optional
        :param preprocessing_func: Function to preprocess images.
        :type preprocessing_func: Callable[[numpy.ndarray], numpy.ndarray], optional
        :param label_encoder: Encoder for label encoding.
        :type label_encoder: sklearn.preprocessing.LabelEncoder or None, optional
        """
        self.parser_func = parser_func
        self.preprocessing_func = preprocessing_func
        self.label_encoder = label_encoder
        self.samples = self._load_dataset_paths(data_dir)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = self.parser_func(image_path)
        image = self.preprocessing_func(image)

        if not torch.is_tensor(image):
            image = torch.tensor(image)

        return image, label

    def _load_dataset_paths(self, data_dir):
        """
        Loads paths of images in the dataset.

        :param str data_dir: Root directory containing the dataset.
        :return: List of tuples containing image paths and their corresponding labels.
        :rtype: List[Tuple[str, int]]
        """
        class_names = os.listdir(data_dir)

        if self.label_encoder is None:
            self.label_encoder = sklearn.preprocessing.LabelEncoder()
            self.label_encoder.fit(class_names)

        samples = []
        for class_name in tqdm(class_names):
            class_data_dir = os.path.join(data_dir, class_name)

            for file_name in os.listdir(class_data_dir):
                samples.append(
                    (
                        os.path.join(class_data_dir, file_name),
                        self.label_encoder.transform([class_name])[0],
                    )
                )

        return samples

In [3]:
from torch.utils.data.sampler import SubsetRandomSampler


dataset = ImageDataset(os.path.join(DATA_DIR, "amazon"))
dataset_size = len(dataset)

indices = list(range(dataset_size))
split = int(np.floor(VAL_SPLIT * dataset_size))

np.random.default_rng(RANDOM_SEED).shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=BATCH_SIZE, sampler=train_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset, batch_size=BATCH_SIZE, sampler=valid_sampler
)

  0%|          | 0/31 [00:00<?, ?it/s]

In [4]:
sample_image, label = dataset[0]
print(sample_image.shape)

torch.Size([1, 3, 300, 300])


In [6]:
print(len(dataset))

2817
